# 大模型量化 (LLM Quantization)

本笔记旨在解释大模型量化精度。

## 1. 什么是大模型量化？

**简单理解**：量化就是给模型“瘦身”。

在深度学习中，模型的**权重（Weights）** 和 **激活值（Activations）** 通常使用 32位浮点数 (FP32)存储。这就好比用一把精度极高的游标卡尺去测量一个物体，虽然很准，但记录的数据很长，占地方。

**量化 (Quantization)** 则是将高精度的浮点数（如 FP32, FP16）转换为低精度的整数（如 INT8, INT4）。这就好比改用一把普通的米尺去测量，虽然精度丢了一点点，但记录的数据变短了，不仅节省显存，还能加快计算速度（整数运算通常比浮点运算快）。

---

## 2. 符号位、指数位、尾数位

要理解量化，首先得知道计算机是怎么存小数的（IEEE 754 标准）。

浮点数公式：
$$ Value = (-1)^S \times 2^{E - Bias} \times (1 + M) $$

一个浮点数由三部分组成：

1.  **符号位 (Sign Bit, S)**: 决定正负。
    *   0 代表正数 (+)，1 代表负数 (-)。
    *   占用 1 bit。
2.  **指数位 (Exponent, E)**: 决定数值的大小范围（Range）。
    *   这就好比科学计数法 $1.23 \times 10^5$ 中的 "5"。
    *   指数位越多，能表示的数越大（或越小）。
3.  **尾数位 (Mantissa / Fraction, M)**: 决定数值的精度（Precision）。
    *   这就好比 $1.234567$ 小数点后面的数字。
    *   尾数位越多，数字越精确，两条刻度线之间分得越细。

**实例：将十进制数 5.5 转换为单精度浮点数（FP32）格式**  
step 1: 将5.5转换为二进制
* 整数部分5：5/2=2余1，2/2=1余0，1/2=0余1，所以整数部分为101
* 小数部分0.5：0.5*2=1，所以小数部分为1
所以$5.5 = 101.1$

step 2: 将二进制数规格化为科学计数法形式
$101.1 = 1.011 \times 2^2$

step 3: 确定各部分的值
* 符号位（S） = 0（因为5.5是正数）所以$S = 0$
* 指数位（E） = 指数为2
* 尾数位（M） = 尾数为011

step 4: 计算偏移指数
* 单精度浮点数的偏移指数（Bias）为127，因此偏移指数=2+127=129

step5：将各部分转换为二进制
* 符号位：0
* 指数位：129 = 10000001
* 尾数位（补零至23位）：01100000000000000000000

step6：合并成完整的32位浮点数
* 合并后的二进制数为：0 10000001 01100000000000000000000  
所以$5.5$的单精度浮点数表示为：0 10000001 01100000000000000000000  

In [1]:
import struct
# 从二进制字符串创建 FP32 浮点数
fp32_bits = '01000000101100000000000000000000'
fp32_bytes = bytes(int(fp32_bits[i:i+8], 2) for i in range(0, 32, 8))
fp32_value = struct.unpack('!f', fp32_bytes)[0]
print(fp32_value)  # 输出: 5.5

5.5


* PS：由于1Byte=8bit，所以32位浮点数占用4Byte内存空间。  
Q: 那么一个 7B（70亿）参数的大模型，如果以 FP16（float16） 精度存储，所需的内存占用是多少？  
A: 70亿个参数，每个参数占用2Byte（16bit），所以总内存占用为14GB。

### 常见格式对比

| 格式 | 总位数 | 符号位 (S) | 指数位 (E) | 尾数位 (M) | 特点 |
| :--- | :--- | :--- | :--- | :--- | :--- |
| **FP32** (单精度) | 32 | 1 | 8 | 23 | 精度高，范围广，训练默认格式。 |
| **FP16** (半精度) | 16 | 1 | 5 | 10 | 显存减半，但指数位少，容易溢出 (Overflow)。 |
| **BF16** (Brain Float) | 16 | 1 | **8** | 7 | Google 提出。**指数位与 FP32 相同**，不易溢出，适合混合精度训练。精度比 FP16 差。 |
| **INT8** (整型) | 8 | 1 (隐式) | 0 | 7 | 没有指数位，只有整数。范围通常是 -128 到 127。 |

**量化的本质**：就是试图用 INT8 这种“粗糙”的刻度，去拟合 FP16 这种“精细”的刻度，同时尽量保证误差在可接受范围内。

## 3. 核心量化原理

最常用的是 **线性量化 (Linear Quantization)**。它假设浮点数 $X_{float}$ 和整数 $X_{int}$ 之间存在线性映射关系。

### 核心公式

$$ X_{int} = \text{round}(\frac{X_{float}}{S} + Z) $$
$$ X_{dequant} = (X_{int} - Z) \times S $$

其中：
- **$S$ (Scale, 缩放因子)**: 类似于步长。决定了两个整数之间代表的浮点距离。
- **$Z$ (Zero-point, 零点)**: 浮点数的 0 对应整数的多少。用于对齐零点。

### 两种主要模式

1.  **对称量化 (Symmetric Quantization)**:
    *   强制 $Z = 0$。
    *   映射范围是对称的（如 -127 到 127）。
    *   公式简化为：$X_{int} = \text{round}(X_{float} / S)$。
    *   **优点**：计算简单，速度快。
    *   **缺点**：如果数据分布不对称（比如 ReLU 激活后全是正数），会浪费一半的整数范围。

2.  **非对称量化 (Asymmetric Quantization)**:
    *   $Z \neq 0$。
    *   可以精确映射数据的最小值 $min$ 到整数的最小值，最大值 $max$ 到整数的最大值。
    *   **优点**：对非对称数据（如 Activation）更准确。
    *   **缺点**：计算时需要加上零点，稍微多一点计算开销。


## 4. 大模型量化方式总结

根据量化发生的时机和对象，主要分为以下几类：

### 按时机分

1.  **PTQ (Post-Training Quantization, 训练后量化)**
    *   模型训练完之后，直接对权重进行量化。不需要重新训练。
    *   **代表算法**: GPTQ, AWQ, LLM.int8(), [SmoothQuant](https://arxiv.org/abs/2211.10438)。
    *   **特点**: 速度快，资源消耗小，目前最主流。

2.  **QAT (Quantization-Aware Training, 量化感知训练)**
    *   在训练过程中就模拟量化带来的误差，让模型学会“适应”低精度。
    *   **特点**: 效果最好，但训练成本高，通常用于从头预训练或深度微调。

### 按对象分

1.  **Weight-Only Quantization (仅权重量化)**
    *   只把模型参数（Weights）量化为 INT4/INT8，计算时解压回 FP16 进行运算。
    *   **目的**: 主要是为了**省显存**，让大模型能跑在消费级显卡上。
    *   **例子**: QLoRA (4-bit), AWQ (4-bit)。

2.  **Activation Quantization (激活值量化)**
    *   不仅权重是整数，中间层的输入输出（Activations）也是整数。
    *   **目的**: 使用整数矩阵乘法单元（如 Tensor Core 的 INT8 模式）来**加速计算**。
    *   **难点**: 激活值主要存在 Outliers（离群点），难以量化（SmoothQuant 解决了这个问题）。

3.  **KV Cache Quantization**
    *   针对长文本推理，KV Cache 会占用大量显存。将其量化为 INT8/FP8。

### 存储占用对比 (以 7B 模型为例)

| 精度 | 每个参数占用 | 7B 模型总显存 (约) | 备注 |
| :--- | :--- | :--- | :--- |
| **FP32** | 4 Bytes | 28 GB | 只有训练时用 |
| **FP16/BF16** | 2 Bytes | 14 GB | 标准推理精度 |
| **INT8** | 1 Byte | 7 GB | 几乎无损 |
| **INT4** | 0.5 Byte | 3.5 GB | 稍微有损，最流行 |


## 5. 写一个最简单的量化器

下面我们用 Python 来实现一个简单的**对称 AbsMax 量化**，看看它是如何工作的。

In [2]:
import torch

def simple_quantize_int8(tensor):
    """
    实现简单的对称 AbsMax 量化 (FP32 -> INT8)
    公式: 
        scale = max(abs(tensor)) / 127
        int8_val = round(tensor / scale)
    """
    # 1. 找到最大绝对值
    max_val = torch.abs(tensor).max()
    
    # 2. 计算缩放因子 Scale
    # INT8 的范围是 [-128, 127]，为了对称通常使用 [-127, 127]
    scale = max_val / 127.0
    
    # 3. 量化：除以 Scale 并四舍五入
    quantized = torch.round(tensor / scale)
    
    # 4. 截断 (Clamp)：防止超出 [-127, 127] 范围
    quantized = torch.clamp(quantized, -127, 127)
    
    # 5. 转换为 int8 类型以节省存储
    quantized_int8 = quantized.to(torch.int8)
    
    return quantized_int8, scale

def dequantize_int8(quantized_int8, scale):
    """
    反量化 (INT8 -> FP32)
    公式: fp32_val = int8_val * scale
    """
    # 转换回 float 进行计算
    return quantized_int8.float() * scale

# --- 测试 ---

# 1. 创建一个模拟权重的随机 Tensor (FP32)
torch.manual_seed(42)
original_weights = torch.randn(5, 5) * 2.0  # 放大一点数值
print("原始数据 (部分):\n", original_weights[:2, :])

# 2. 量化
q_weights, scale = simple_quantize_int8(original_weights)
print("\n量化后的数据 (INT8) (部分):\n", q_weights[:2, :])
print(f"Scale (缩放因子): {scale.item():.6f}")

# 3. 反量化
reconstructed_weights = dequantize_int8(q_weights, scale)
print("\n反量化后的数据 (FP32) (部分):\n", reconstructed_weights[:2, :])

# 4. 计算误差 (量化损失)
error = torch.abs(original_weights - reconstructed_weights).mean()
print(f"\n平均绝对误差 (MAE): {error.item():.6f}")

# 5. 验证存储占用
orig_mem = original_weights.element_size() * original_weights.numel()
quant_mem = q_weights.element_size() * q_weights.numel()
print(f"\n原始大小: {orig_mem} bytes")
print(f"量化大小: {quant_mem} bytes (压缩率 {orig_mem/quant_mem:.1f}x)")

原始数据 (部分):
 tensor([[ 3.8538,  2.9746,  1.8014, -4.2110,  1.3568],
        [-2.4691, -0.0861, -3.2093, -1.5043, -1.3732]])

量化后的数据 (INT8) (部分):
 tensor([[ 106,   82,   49, -115,   37],
        [ -68,   -2,  -88,  -41,  -38]], dtype=torch.int8)
Scale (缩放因子): 0.036487

反量化后的数据 (FP32) (部分):
 tensor([[ 3.8676,  2.9919,  1.7879, -4.1960,  1.3500],
        [-2.4811, -0.0730, -3.2109, -1.4960, -1.3865]])

平均绝对误差 (MAE): 0.008797

原始大小: 100 bytes
量化大小: 25 bytes (压缩率 4.0x)


### 结果分析

运行上面的代码，你会发现：
1.  **数据变了**：原始的 `1.9269` 可能变成了 `1.9213`。这就是精度损失。
2.  **大小变了**：存储空间变成了原来的 1/4 (32bit -> 8bit)。
3.  **Scale 很重要**：它连接了整数世界和浮点世界。

在实际的大模型量化算法（如 AWQ）中，会比这个复杂得多，比如会按通道（Per-Channel）或按组（Per-Group）来计算 Scale，以减少误差。